## Reducer: Adam Kline

## Reviewer: (Put your name here)

# jupyter notebook crash course

Click on a code cell (has grey background) then press Shift-Enter (at the same time) to run a code cell. That will add the controls (buttons, etc) you use to do the reduction one-by-one; then use them for reduction.

# reducer crash course

### Rule 0: Run the code cells in order

The world won't end if you break this rule, but you are more likely to end up with nonsensical results or errors. Incidentally, welcome to python indexing, which starts numbering at zero.

### Rule 1: Do not run this notebook in the directory containing your unreduced data

`reducer` will not overwrite anything but the idea is that you will keep a copy of this notebook with your reduced data.

### Rule 2: Keep the cells you need, delete the ones you don't

IPython notebooks are essentially customizable apps. If you don't shoot dark frames, for example, delete the stuff related to darks.

### Rule 3: If you find bugs, please report them

You can report bugs, make feature requests or (best of all) submit pull requests from [reducer's home on github](https:github.com/mwcraig/reducer)

### Bonus Pro Tip: Feel free to ignore the code in the code cells

Code is there so that people who know python can see what is going on, but if you don't know python you should still be able to use the notebook. Just remember to Shift-Enter on each code cell to run it, then fill in the form(s) that appear in the notebook.


In [1]:
from __future__ import division, print_function

import reducer.gui
import reducer.astro_gui as astro_gui
from reducer.image_browser import ImageBrowser

from ccdproc import ImageFileCollection

from reducer import __version__
print(__version__)

0.4.1


# Enter name of directory that contains your data in the cell below, or...

### ...leave it unchanged to try out reducer with low-resolution dataset

That low-resolution dataset will expand to about 300MB when uncompressed


In [2]:
# To use the sample data set:
#data_dir = reducer.notebook_dir.get_data_path()

# Or, uncomment line below and modify as needed
data_dir = '../2018-07-23'

destination_dir = '.'

# Type any comments about this dataset here

Double-click on the cell to start editing it.

# Load your data set

In [3]:
images = ImageFileCollection(location=data_dir, keywords='*')

# Image Summary

## Includes browser and image/metadata viewer

This is not, strictly speaking, part of reduction, but is a handy way to take a quick look at your files.

In [4]:
fits_browser = ImageBrowser(images, keys=['imagetyp', 'exposure'])
fits_browser.display()

ImageBrowser(children=(Accordion(children=(Accordion(children=(Box(children=(Select(layout=Layout(width='100%'…

### You can reconfigure the image browser if you want (or not)

By passing different keys into the tree constructor you can generate a navigable tree based on any keys you want.

In [5]:
im_a_tree_too = ImageBrowser(images, keys=['filter', 'imagetyp', 'exposure'])
im_a_tree_too.display()

ImageBrowser(children=(Accordion(children=(Accordion(children=(Accordion(children=(Box(children=(Select(layout…

# Make a master bias image

## Reduce the bias images

In [6]:
bias_reduction = astro_gui.Reduction(description='Reduce bias frames',
                                     toggle_type='button',
                                     allow_bias=False,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     apply_to={'imagetyp': 'bias'},
                                     destination=destination_dir)
bias_reduction.display()

Reduction(children=(Box(children=(ToggleButton(value=False, description='Reduce bias frames'),)), VBox(childre…

Reduce bias frames Yes
	Copy without any other action? No
	Subtract overscan? No
	Trim (specify region to keep)? Yes
		Region is all of 0  and along the other axis from   0  up to (but not including):  4096


## Combine bias images to make master bias

In [7]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
bias_settings = astro_gui.Combiner(description="Master Bias Settings",
                                   toggle_type='button',
                                   file_name_base='master_bias',
                                   image_source=reduced_collection,
                                   apply_to={'imagetyp': 'bias'},
                                   destination=destination_dir)
bias_settings.display()

Combiner(children=(Box(children=(ToggleButton(value=False, description='Master Bias Settings'),)), VBox(childr…

Master Bias Settings Yes
	Clip before combining? Yes
		Clip by min/max? No
		Sigma clip? Yes
			Low threshold: 5.0
			High threshold: 5.0
	Combine images? Yes
		Combination method:: Average
		Scale before combining? No
	Group by: No


# Make a master dark

## Reduce dark images

In [8]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
dark_reduction = astro_gui.Reduction(description='Reduce dark frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'dark'})

dark_reduction.display()

Reduction(children=(Box(children=(ToggleButton(value=False, description='Reduce dark frames'),)), VBox(childre…

Reduce dark frames Yes
	Copy without any other action? No
	Subtract overscan? No
	Trim (specify region to keep)? Yes
		Region is all of 0  and along the other axis from   0  up to (but not including):  4096
	Subtract bias? Yes
		Source: notebook


## Combine reduced darks to make master(s)

Note the ``Group by`` option in the controls that appear after you execute the cell below. `reducer` will make a master for each value of the FITS keyword listed in ``Group by``. By default this keyword is named ``exposure`` for darks, so if you have darks with exposure times of 10 sec, 15 sec and 120 sec you will get three master darks, one for each exposure time.

In [9]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

dark = astro_gui.Combiner(description="Make Master Dark",
                          toggle_type='button',
                          file_name_base='master_dark',
                          group_by='exposure',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'dark'},
                          destination=destination_dir)
dark.display()

Combiner(children=(Box(children=(ToggleButton(value=False, description='Make Master Dark'),)), VBox(children=(…

Make Master Dark Yes
	Clip before combining? Yes
		Clip by min/max? No
		Sigma clip? Yes
			Low threshold: 5.0
			High threshold: 5.0
	Combine images? Yes
		Combination method:: Average
		Scale before combining? Yes
			Which should scale to same value?: mean
	Group by: Yes
		Keywords (comma-separated): exposure


# Make master flats

## Reduce flat images

In [10]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
flat_reduction = astro_gui.Reduction(description='Reduce flat frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=True,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'flat'})

flat_reduction.display()

Reduction(children=(Box(children=(ToggleButton(value=False, description='Reduce flat frames'),)), VBox(childre…

Reduce flat frames Yes
	Copy without any other action? No
	Subtract overscan? No
	Trim (specify region to keep)? Yes
		Region is all of 0  and along the other axis from   0  up to (but not including):  4096
	Subtract bias? Yes
		Source: notebook
	Subtract Dark? Yes
		Source: notebook
		Scale dark by exposure time (if needed): True


## Build masters by combining

Again, note the presence of ``Group by``. If you typically use twilight flats you will almost certainly want to group by filter, not by filter and exposure.

In [11]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

flat = astro_gui.Combiner(description="Make Master Flat",
                          toggle_type='button',
                          file_name_base='master_flat',
                          group_by='exposure, filter',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'flat'},
                          destination=destination_dir)
flat.display()

Combiner(children=(Box(children=(ToggleButton(value=False, description='Make Master Flat'),)), VBox(children=(…

Make Master Flat Yes
	Clip before combining? Yes
		Clip by min/max? No
		Sigma clip? Yes
			Low threshold: 5.0
			High threshold: 5.0
	Combine images? Yes
		Combination method:: Average
		Scale before combining? Yes
			Which should scale to same value?: mean
	Group by: Yes
		Keywords (comma-separated): filter


# Reduce the science images

There is some autmatic matching going on here:

+ If darks are subtracted a dark of the same edxposure time will be used, if available. If not, and dark scaling is enabled, the dark with the closest exposure time will be scaled to match the science image.
+ If the dark you want to scale appears not to be bias-subtracted an error will be raised.
+ Flats are matched by filter.

In [7]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
light_reduction = astro_gui.Reduction(description='Reduce light frames',
                                      toggle_type='button',
                                      allow_cosmic_ray=True,
                                      master_source=reduced_collection,
                                      input_image_collection=images,
                                      destination=destination_dir,
                                      apply_to={'imagetyp': 'light'})

light_reduction.display()

Reduction(children=(Box(children=(ToggleButton(value=False, description='Reduce light frames'),)), VBox(childr…

Reduce light frames Yes
	Copy without any other action? No
	Subtract overscan? No
	Trim (specify region to keep)? Yes
		Region is all of 0  and along the other axis from   0  up to (but not including):  4096
	Subtract bias? Yes
		Source: notebook
	Subtract Dark? Yes
		Source: notebook
		Scale dark by exposure time (if needed): True
	Flat correct? Yes
		Source: notebook
	Clean cosmic rays? No


## Wonder what the reduced images look like? Make another image browser...

In [8]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

In [9]:
reduced_browser = ImageBrowser(images, keys=['imagetyp', 'exposure'])
reduced_browser.display()

TypeError: unhashable type: 'MaskedConstant'